In [103]:
# python -m spacy download de_core_news_sm


In [105]:
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
import pandas as pd
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


try:
    nlp_en = spacy.load("en_core_web_sm")
except Exception as e:
    print("Please install the spaCy English model: en_core_web_sm")

try:
    nlp_de = spacy.load("de_core_news_sm")
except Exception as e:
    print("Please install the spaCy German model: de_core_news_sm")


In [106]:
pd.set_option('display.max_columns', None)


In [107]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


def get_value_counts_for_category(dataframe, column, timing):
    s = ""
    for pair in list(dataframe[column].value_counts(dropna=False).items()):
        s += str(pair) + " "
    print(f"{column}: {timing} change - {s}")


def preprocess_is_registered_with_afa(dataframe):
    get_value_counts_for_category(dataframe, "registered_with_afa", "Before")
    
    print(f"registered_with_afa: Before filtering out odd options -{len(dataframe)}")
    dataframe = dataframe[dataframe['registered_with_afa'].isin(["yes", "not yet"])].copy()
    print(f"registered_with_afa: After filtering out odd options -{len(dataframe)}")
    
    get_value_counts_for_category(dataframe, "registered_with_afa", "After")
    
    return dataframe


def preprocess_is_meeting_with_advisor(dataframe):
    
    meeting_with_advisor_rank = {"I haven't requested my next meeting yet.": 1,
                                 "Waiting to hear back for over two weeks": 2,
                                 "Yes, I have a scheduled date for my next meeting.": 3,
                                 "Yes, I've already had a meeting about the Bildungsgutschein.": 4,
                                 "Yes, I've requested a meeting but haven't received a date yet.": 5,
                                 "Not yet": 6,
                                 "Yes, I have had a meeting with a job advisor": 7,
                                 "Yes, I am waiting for a meeting with a job advisor": 8}
    dataframe['feat_in_contact_with_job_advisor'] = dataframe['in_contact_with_job_advisor'].map(meeting_with_advisor_rank).fillna(-1)
    
    
    return dataframe


def preprocess_languages(dataframe):
    def determine_communication_language(row):
        """
		Determine the communication language based on the given conditions:
		1. If 'communication_language' is not missing, use its value.
		2. Otherwise, compare 'english_level' and 'german_level':
		   - If 'german_level' is higher, return 'German'.
		   - If 'english_level' is higher, return 'English'.
		   - If both are equal, return 'Bilingual'.
		"""
        
        bilingual_condition_1 = row['english_level'] >= "B1"
        bilingual_condition_2 = row['german_level'] >= "B1"
        
        if pd.notna(row['communication_language']):
            return row['communication_language']
        elif row['german_level'] > row['english_level']:
            return 'German'
        elif row['english_level'] > row['german_level']:
            return 'English'
        if bilingual_condition_1 and bilingual_condition_2:
            return "Bilingual"
        elif row['english_level'] == "Unknown" and row['german_level'] != "Unknown":
            return 'German'
        elif row['german_level'] == "Unknown" and row['english_level'] != "Unknown":
            return 'English'
        
        return 'Unknown'
    
    language_categories = ["Unknown", "A1", "A2", "B1", "B2", "C1", "C2"]
    
    # Dictionary mapping each term to a unified proficiency level
    proficiency_mapping = {
            'mother tongue': 'C2',
            'muttersprache': 'C2',  # German for 'mother tongue'
            'native': 'C2',
            'fluent': 'C1',
            'proficient': 'C1',
            'advanced': 'C1',
            'upper-intermediate': 'B2',
            'intermediate': 'B1',
            'pre-intermediate': 'A2',
            'beginner': 'A1',
            "i'm not sure": 'Unknown',
            np.nan:"Unknown",
            None: 'Unknown',
                  "Unknown":"Unknown",
    }
    dataframe['english_level'] = dataframe['english_level'].fillna("Unknown")
    dataframe['german_level'] = dataframe['german_level'].fillna("Unknown")
    
    english_levels = dataframe['english_level'].str.strip().str.split(" - ", expand=True)
    if len(english_levels.columns)>1:
        dataframe['english_level'] = english_levels[1].fillna(english_levels[0])
    else:
        dataframe['english_level'] = english_levels

    german_level = dataframe['german_level'].str.strip().str.split(" - ", expand=True)
    if len(german_level.columns)>1:
        dataframe['german_level'] = german_level[1].fillna(german_level[0])
    else:
        dataframe['german_level'] = german_level

    dataframe['german_level'] = dataframe['german_level'].map(proficiency_mapping)
    dataframe['english_level'] = dataframe['english_level'].map(proficiency_mapping)
    if dataframe['german_level'].isna().sum()>1:
        print(1)
    dataframe['german_level'] = pd.Categorical(dataframe['german_level'], categories=language_categories, ordered=True)
    dataframe['english_level'] = pd.Categorical(dataframe['english_level'], categories=language_categories,
                                                ordered=True)
    
    dataframe['communication_language'] = dataframe.apply(determine_communication_language, axis=1)
    
    return dataframe


def preprocess_visa_status(dataframe):
    
    def normalize_status(current_status):
        if pd.isna(current_status):
            return ":unknown"
        
        working_status = str(current_status).lower().split("(")[0]
        if working_status.find("eu citi") > -1:
            return "eu citizen"
        if working_status.find("ständiger wohnsitz") > -1 or working_status.find("permanent resident") > -1:
            return "permanent resident"
        if working_status.find("eu blue card") > -1 or working_status.find("blaue karte") > -1:
            return "eu blue card"
        if working_status.find("24") > -1:
            return "paragraph 24"
        if working_status.find("asylum") > -1 or working_status.find("24") > -1:
            return "asylum seeker"
        if working_status.find("temporary stay") > -1 or working_status.find("aufenthaltserlaubnis") > -1:
            return "temporary stay permit"
        
        if working_status.find("work visa") > -1:
            return "work visa"
        
        if working_status.find("student visa") > -1:
            return "student visa"
        if working_status.find("famil") > -1:
            return "family reunification"
        
        if working_status.find("job seeker") > -1:
            return "job seeker visa"
        
        if working_status.find("other") > -1:
            return ":other"
        return ":other"
    
    dataframe['feat_visa_status'] = dataframe['status'].apply(normalize_status)
    
    return dataframe


def preprocess_mql_date_features(dataframe, regular=True):
    if regular:
        dataframe['feat_day_of_mql'] = dataframe['mql_date'].dt.strftime("%A")
        dataframe['feat_mql_month_name'] = dataframe['mql_date'].dt.month_name()
    else:
        dataframe["feat_weekday"] = dataframe["mql_date"].dt.weekday  # Monday = 0, ..., Sunday = 6
        dataframe["feat_weekday_sin"] = np.sin(2 * np.pi * dataframe["weekday"] / 7)
        dataframe["feat_weekday_cos"] = np.cos(2 * np.pi * dataframe["weekday"] / 7)

        dataframe['feat_mql_month_name'] = dataframe['mql_date'].dt.month_name()
        dataframe = dataframe.drop('weekday', axis=1)
        
    return dataframe


def preprocess_mql_hour_minutes_features(dataframe):
    dataframe['mql_hour'] = dataframe['mql_date'].dt.hour
    dataframe['mql_minutes'] = dataframe['mql_date'].dt.minute
    dataframe['mql_week_number'] = dataframe['mql_date'].dt.isocalendar().week
    
    dataframe["feat_mql_hour_sin"] = np.sin(2 * np.pi * dataframe['mql_hour'] / 24)
    dataframe["feat_mql_hour_cos"] = np.cos(2 * np.pi * dataframe['mql_hour'] / 24)
    
    dataframe["feat_mql_minute_sin"] = np.sin(2 * np.pi * dataframe['mql_minutes'] / 60)
    dataframe["feat_mql_minute_cos"] = np.cos(2 * np.pi * dataframe['mql_minutes'] / 60)
    
    dataframe['feat_mql_week_number_sin'] = np.sin(2 * np.pi * dataframe['mql_week_number'] / 52)
    dataframe['feat_mql_week_number_cos'] = np.cos(2 * np.pi * dataframe['mql_week_number'] / 52)
    
    dataframe = dataframe.drop(["mql_hour", "mql_minutes", "mql_week_number"], axis=1)
    
    return dataframe


def preprocess_days_to_mql(dataframe, more_than_a_day_feature=False):
    dataframe['days_to_mql'] = (dataframe['mql_date'] - dataframe['created_at']).dt.days
    
    if more_than_a_day_feature:
        dataframe['more_than_a_day'] = dataframe['days_to_mql']>1
    
    dataframe['days_to_mql'] = np.log1p(dataframe['days_to_mql'])
    
    return dataframe



def preprocess_time_to_mql(dataframe, time_units="seconds"):
    """Returns a log version of the time to MQL - either seconds or minutes"""
    dataframe['time_to_mql'] = np.log((dataframe['mql_date'] - dataframe['created_at']).dt.seconds)
    
    if time_units == "minutes":
        dataframe['time_to_mql'] = np.log1p((dataframe['mql_date'] - dataframe['created_at']).dt.seconds // 60)
    
    return dataframe


def preprocess_age_ranges(dataframe):
    get_value_counts_for_category(dataframe, "age_range", "Before")
    dataframe = dataframe.dropna(subset=["age_range"]).copy()
    dataframe = dataframe[~dataframe['age_range'].str.contains(";")].copy()
    
    dataframe['age_range'] = dataframe['age_range'].replace({"32": "25-40"})
    dataframe['age_range'] = np.where(dataframe['age_range'] == '16-22', "16-21", dataframe['age_range'])

    ordinal_ages = {'16-21': 1,
     '22-24': 2,
     '25-40': 3,
     '41-55': 4,
     '56-64': 5,
     '65+': 6}

    # dataframe['age_range'] = pd.Categorical(dataframe['age_range'],
    #                                         categories=['16-21', '22-24', '25-40', '41-55', '56-64', '65+'],
    #                                         ordered=True)

    dataframe['feat_age_range'] = dataframe['age_range'].map(ordinal_ages)
    
    return dataframe


def preprocess_employment_status(dataframe):
    get_value_counts_for_category(dataframe, 'employment_status', "Before")
    
    dataframe['employment_status'] = dataframe['employment_status'].fillna("unknown")
    
    def norm_employment(status):
        status_str = str(status).lower()
        if status_str.find("unempl") > -1:
            return "unemployed"
        if status_str.find("self") > -1 or status_str.find("freel") > -1:
            return "freelance"
        if status_str.find("employed") > -1:
            return "employed"
        if status_str.find("train") > -1 or status_str.find("azub") > -1:
            return "trainee"
        if status_str.find("mini") > -1 or status_str.find("part") > -1:
            return "trainee"
        if status_str.find("student") > -1:
            return status_str
        if status_str.find("appre") > -1:
            return "apprentice"
        return "other"
    
    dataframe['employment_status'] = dataframe['employment_status'].apply(norm_employment)
    
    get_value_counts_for_category(dataframe, 'employment_status', "After")
    
    return dataframe


def preprocess_utm_source(dataframe):
    get_value_counts_for_category(dataframe, "utm_source", "Before")
    dataframe['utm_source'] = dataframe['utm_source'] = dataframe['utm_source'].fillna("unknown source")
    dataframe['utm_source'] = np.where(dataframe['utm_source'].str.contains("source"), "source",
                                       dataframe['utm_source'])
    get_value_counts_for_category(dataframe, "utm_source", "After")
    
    return dataframe


def preprocess_all_data_ad_name(dataframe):
    dataframe['ad_name'] = dataframe['ad_name'].str.lower().str.strip()
    pattern = re.compile(r'\[(?P<key>type|usp|prog|3s):(?P<value>[^\]]+)\]')

    def normalizing_ad_type(type_):
        if pd.isna(type_):
            return "not specified"
        if "mage" in type_:
            return "image"
        if "carou" in type_:
            return "carousel"
        if "vid" in type_:
            return "video"
        return type_

    def program_in_ad_imputer(program):
        if pd.isna(program):
            return "not specified"
        if 'data' in program or program == 'date':
            return 'da'
        if program.startswith("_") or program.startswith(":"):
            return program[1:].replace("-", " ")

        return program.replace("-", " ")
    def extract_selected_keys(ad_name):
        """Extract type, usp, and prog from the ad_name string and return them as a dictionary."""
        matches = pattern.finditer(ad_name)
        return {m.group('key'): m.group('value').strip('_') for m in matches}

    # Apply function to ad_name column and create a DataFrame from the dictionaries
    extracted_df = dataframe['ad_name'].apply(lambda x: pd.Series(extract_selected_keys(x), dtype='object'))
    extracted_df['usp'] = extracted_df['usp'].fillna("not specified")
    extracted_df['3s'] = extracted_df['3s'].fillna("not specified")
    extracted_df['type'] = extracted_df['type'].fillna("not specified").apply(normalizing_ad_type)
    extracted_df['prog'] = extracted_df['prog'].fillna("not specified").apply(program_in_ad_imputer)
    # Join the new columns to the original DataFrame
    dataframe = dataframe.join(extracted_df)

    return dataframe


In [108]:
new_df = pd.read_csv('base_data.tsv', sep="\t", index_col='id')

C:\Users\Tal\AppData\Local\Temp\ipykernel_23620\281279912.py:1: DtypeWarning: Columns (3,6,12,18,26,36) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv('base_data.tsv', sep="\t", index_col='id')


In [109]:
new_df.columns = [x.split("bg0__")[1] if "bg0__" in x else x for x in [x.replace("bg__", "") .split("properties_")[1] if "properties_" in x else x for x in new_df.columns ]]

In [111]:
new_df.loc[7420801]

age_range                                                                               NaN
employment_situation                                                             Unemployed
german_job_permit                                                                       NaN
english_level                                                                 C1 - Advanced
german_level                                                        B2 - Upper-intermediate
form_language                                                                           NaN
highest_level_of_education                                 Ausbildung / Vocational training
how_did_you_hear_about_us                                                     Search engine
in_contact_with_job_advisor                                                         Not yet
last_job                                                                            Factory
preferred_cohort                                                              No

In [112]:
def norm_employment(status):
    status_str = str(status).lower().strip()
    unemployed_cond = [status_str.find("unempl") > -1, status_str.find("wife")>-1
                       ,status_str.find("leave")>-1, status_str.find("ausfrau")>-1,
                       status_str.find('elternzeit')>-1, status_str.find("home")>-1, status_str.find('mom')>-1,
                      status_str.find('mother')>-1,status_str.find('retired')>-1,]
    if any(unemployed_cond):
        return "unemployed"
    if status_str.find("self") > -1 or status_str.find("freel") > -1 or status_str.find("selbstst")>-1:
        return "freelance"
    if status_str.find("employed") > -1 or status_str.find('pair')>-1 or status_str.find("worker")>-1:
        return "employed"
    if status_str.find("train") > -1 or status_str.find("azub") > -1 or status_str.find("appre") > -1:
        return "trainee"
    if status_str.find("mini") > -1 or status_str.find("part") > -1 or status_str.find("intern") > -1:
        return "trainee"
    if status_str.find("student") > -1:
        return "student"
    if status_str.find("refugee")>-1 or status_str.find("asyl")>-1 or status_str.find("24")>-1:
        return "refugee"
    if status_str.find("usbildung") > -1:
        return "training"
    if status_str=="sonstiges" or status_str=="other":
        return "other"
    return "other"
    
    dataframe['employment_status'] = dataframe['employment_status'].apply(norm_employment)
    return dataframe


def preprocess_age_ranges(dataframe):
    get_value_counts_for_category(dataframe, "age_range", "Before")
    dataframe = dataframe.dropna(subset=["age_range"]).copy()
    dataframe = dataframe[~dataframe['age_range'].str.contains(";")].copy()
    
    dataframe['age_range'] = dataframe['age_range'].replace({"32": "25-40"})
    dataframe['age_range'] = np.where(dataframe['age_range'] == '16-22', "16-21", dataframe['age_range'])

    ordinal_ages = {'16-21': 1,
     '22-24': 2,
     '25-40': 3,
     '41-55': 4,
     '56-64': 5,
     '65+': 6}

    dataframe['feat_age_range'] = dataframe['age_range'].map(ordinal_ages)
    get_value_counts_for_category(dataframe, "age_range", "After")
    return dataframe


def clean_preferred_cohort(cohort_series):
    # Standardize German months to English
    month_mapping = {
        "Januar": "January",
        "Februar": "February",
        "März": "March",
        "Mai": "May",
        "Juni": "June",
        "Juli": "July",
        "Oktober": "October",
        "Dezember": "December"
    }
    
    # Replace German month names
    cohort_series = cohort_series.replace(month_mapping, regex=True)

    # Remove words like "Class" and extra spaces
    cohort_series = cohort_series.str.replace(r' Class', '', regex=True).str.strip()
    
    # Fix abbreviated months (e.g., 'Jan 2025' -> 'January 2025')
    cohort_series = cohort_series.replace({
        "Jan ": "January ",
        "Feb ": "February ",
        "Mar ": "March ",
        "Apr ": "April ",
        "Jun ": "June ",
        "Jul ": "July ",
        "Aug ": "August ",
        "Sep ": "September ",
        "Oct ": "October ",
        "Nov ": "November ",
        "Dec ": "December "
    }, regex=True)
    
    # Convert to datetime, forcing errors to NaT for invalid entries
    return pd.to_datetime(cohort_series, format='%B %Y', errors='coerce').dt.tz_localize('UTC')


def preprocess_registered_with_the_jobcenter(val):
    
    if pd.isna(val):
        return "not specified"
    if val.lower().strip()=="yes":
        return "yes"
    return "not yet"


In [113]:
df = new_df.copy()

In [114]:
df['createdate'] = pd.to_datetime(df['createdate'])
df['requested_bg_date'] = pd.to_datetime(df['requested_bg_date'])
df['mql_date'] = pd.to_datetime(df['mql_date'])
df['sql_date'] = pd.to_datetime(df['sql_date'])
df['bg_enrolled_date'] = pd.to_datetime(df['bg_enrolled_date'])



In [115]:
df = preprocess_age_ranges(df)
df['feat_employment_situation'] =  df['employment_situation'].apply(norm_employment)
df['feat_german_job_permit'] = df['german_job_permit'].str.lower().fillna("not specified")
df['feat_finished_completing_profile'] = ~df['why_do_you_want_to_start_a_career_in_tech'].isna()
started_completing_profile = ~df['last_job'].isna() | ~df['time_worked_in_de'].isna() | ~df['highest_level_of_education'].isna() | ~df['state'].isna() | ~df['how_did_you_hear_about_us'].isna() | ~df['why_do_you_want_to_start_a_career_in_tech'].isna() 
df['feat_started_completing_profile'] = started_completing_profile
df['feat_completing_profile_rate'] = df[['last_job','time_worked_in_de','highest_level_of_education','state' , 'how_did_you_hear_about_us', 'why_do_you_want_to_start_a_career_in_tech']].notnull().sum(axis=1)/6

age_range: Before change - ('25-40', 62009) ('41-55', 22452) ('22-24', 9443) ('56-64', 4466) (nan, 2844) ('16-21', 1240) ('65+', 211) ('16-22', 11) ('22-24;16-21', 3) ('22-24;25-40', 3) ('22-25;26-40', 2) ('32', 1) 
age_range: After change - ('25-40', 62010) ('41-55', 22452) ('22-24', 9443) ('56-64', 4466) ('16-21', 1251) ('65+', 211) 


In [116]:
complete = ['last_job','time_worked_in_de','highest_level_of_education','state' , 'how_did_you_hear_about_us', 'why_do_you_want_to_start_a_career_in_tech']

for col in complete:
    df[f"feat_completed_{col}"] = df[col].isnull()

In [117]:
eng_level_mapping = {
    # A1-A2 category
    'a1-a2': 1,
    'A1 - Beginner': 1,
    "A2 - Beginner":1,
    "Beginner":1,
    "A2": 1,
    'A1-A2': 1,
     'a2': 1,
    'A2 - Pre-intermediate': 1,
    ' A1 - Beginner': 1 ,

    # B1-B2 category
    'b1-b2': 2,
    'B1 - Intermediate': 2,
    'B1-B2': 2,
    'B2 - Upper-intermediate': 2,
    'Intermediate': 2,

    # C1-C2 category
    'c1-c2': 3,
    'C1 - Advanced': 3,
    'C2 - Proficient': 3,
    "C1":3,
    'C1-C2': 3,
    'Advanced': 3,
    "Native":3,
    'Fluent': 3,

    # Not sure category
    "I'm not sure": -1,
    "unknown":-1,
    "Muttersprache":3 ,
    'Mother Tongue': 3,
    'mother tongue': 3,
    'Mother tongue': 3,
    "0": -1,
    "English":-1,
    np.nan: -1,
}

df['english_level_pat'] = df['english_level'].str.strip().str.extract(r"\((.*?)\)")[0].fillna(df['english_level'])
df['feat_eng_level_grouped'] = df['english_level_pat'].map(eng_level_mapping)

df['german_level_pat'] = df['german_level'].str.strip().str.extract(r"\((.*?)\)")[0].fillna(df['german_level'])
df['feat_german_level_grouped'] = df['german_level_pat'].str.strip().map(eng_level_mapping)
df['feat_form_language'] = df['form_language'].fillna("not specified")

df['feat_bilingual'] = df[['feat_german_level_grouped','feat_eng_level_grouped']].sum(axis=1)>=5

In [118]:
education_mapping = {
    # No Formal Education
    None: "No Formal Education",
    "None": "No Formal Education",

    # Secondary Education
    "High school diploma": "Secondary Education",
    "Secondary school": "Secondary Education",
    "High school": "Secondary Education",
    "Mittlere Reife": "Secondary Education",
    "MSA": "Secondary Education",
    "Realschule": "Secondary Education",
    "Mittelschule": "Secondary Education",
    "mittlere reife": "Secondary Education",
    "Mittelschulabschluss": "Secondary Education",
    "Quali": "Secondary Education",
    "A levels": "Secondary Education",
    "Förderschule": "Secondary Education",
    
    # Vocational Training
    "Ausbildung / Vocational training": "Vocational Training",
    "Ausbildung/Vocational training": "Vocational Training",
    "Techniker": "Vocational Training",
    "Meister": "Vocational Training",
    "Fachwirt": "Vocational Training",
    "Staatlich geprüfter Techniker": "Vocational Training",
    
    # Lower-Tier Diplomas
    "Hauptschule": "Secondary Education",
    "Haupt": "Secondary Education",
    "hauptschule": "Secondary Education",
    "Hauptschulabschluss": "Secondary Education",
    "Hauptschulabschluß": "Secondary Education",
    "Qualifizierter Hauptschulabschluss": "Secondary Education",
    "Erweiterter Hauptschulabschluss": "Secondary Education",
    "HA10": "Secondary Education",
    
    # Undergraduate Degree
    "Bachelor degree": "Undergraduate Degree",
    "Bachelor's degree": "Undergraduate Degree",
    "Bachelor": "Undergraduate Degree",
    "University": "Undergraduate Degree",
    "Undergraduate": "Undergraduate Degree",
    "Hochschule": "Undergraduate Degree",
    "Diploma": "Undergraduate Degree",
    "Diplom (FH)": "Undergraduate Degree",
    "Associate degree": "Undergraduate Degree",
    
    # Graduate Degree
    "Master degree": "Graduate Degree",
    "Master's degree": "Graduate Degree",
    "Master of Business Administration": "Graduate Degree",
    "Diplom": "Graduate Degree",
    "Diplom Ingenieur": "Graduate Degree",
    "Fachhochschulreife": "Graduate Degree",
    "Fachabitur": "Graduate Degree",
    "Staatsexamen": "Graduate Degree",
    
    # Doctorate
    "Doctorate": "Doctorate",
    
    # Other (Unclear / Edge Cases)
    "Driver": "Other"
    
}



df["feat_education_level"] = df["highest_level_of_education"].map(education_mapping).str.lower().fillna("not specified")


In [120]:
df.head()



KeyError: 7420801

In [20]:
df = preprocess_is_meeting_with_advisor(df)
df["last_job"] = df["last_job"].str.replace("Hotal/Tourism","Hotel/Tourism").str.lower().fillna("not specified")
df['feat_last_job_related'] = df['last_job'].isin(["it","it branche", "web development","marketing"])
df['preferred_cohort'] = clean_preferred_cohort(df['preferred_cohort'])
df['feat_expectation'] = (df['preferred_cohort'] - df['mql_date']).dt.days
df['feat_desired_program_length'] = df['program_length'].fillna("not specified")
df['feat_registered_with_the_jobcenter'] = df['registered_with_the_jobcenter'].apply(preprocess_registered_with_the_jobcenter)


In [21]:
df.head()

,age_range,employment_situation,german_job_permit,english_level,german_level,form_language,highest_level_of_education,how_did_you_hear_about_us,in_contact_with_job_advisor,last_job,preferred_cohort,program_length,registered_with_the_jobcenter,state,status,time_worked_in_de,field_of_interest,field_of_interest_2,residents,createdate,requested_bg_date,requested_bg,mql_date,sql_date,got_sql,bg_enrolled_date,lpvariant,utm_campaign,utm_content,utm_id,utm_medium,utm_source,utm_term,students_time_to_graduation,why_do_you_want_to_start_a_career_in_tech,closed_won_deal__program_duration,feat_age_range,feat_employment_situation,feat_german_job_permit,feat_finished_completing_profile,feat_started_completing_profile,feat_completing_profile_rate,feat_completed_last_job,feat_completed_time_worked_in_de,feat_completed_highest_level_of_education,feat_completed_state,feat_completed_how_did_you_hear_about_us,feat_completed_why_do_you_want_to_start_a_career_in_tech,english_level_pat,feat_eng_level_grouped,german_level_pat,feat_german_level_grouped,feat_form_language,feat_bilingual,feat_education_level,feat_in_contact_with_job_advisor,feat_last_job_related,feat_expectation,feat_desired_program_length,feat_registered_with_the_jobcenter
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
295243348,25-40,Employed,NaN,intermediate (b1-b2),advanced (c1-c2),NaN,Bachelor degree,Search engine,Not yet,it,2024-04-01 00:00:00+00:00,NaN,Yes,Baden-Württemberg,EU Blue Card,NaN,Data Analytics,NaN,NaN,2024-03-18 16:56:25.263000+00:00,NaT,not requested,2024-03-18 17:01:38.402000+00:00,NaT,no sql,NaT,NaN,20374398137,300016462,20374398137,email,hs_email,masters data analytics online,NaN,I am already in tech. I just want to learn a n...,NaN,3,employed,not specified,True,True,0.833333,False,True,False,False,False,False,b1-b2,2,c1-c2,3,not specified,True,undergraduate degree,6.0,True,13.0,not specified,yes
295243539,22-24,Unemployed,NaN,advanced (c1-c2),beginner (a1-a2),NaN,Bachelor degree,Search engine,NaN,education,2024-04-01 00:00:00+00:00,NaN,Not yet,Hessen,eu citizen,NaN,Cybersecurity,NaN,NaN,2024-03-18 16:54:41.915000+00:00,NaT,not requested,2024-03-18 16:59:00.319000+00:00,NaT,no sql,NaT,NaN,531020879,84319356864966,531020879,paid,bing,learn cyber security,NaN,NaN,NaN,2,unemployed,not specified,False,True,0.666667,False,True,False,False,False,True,c1-c2,3,a1-a2,1,not specified,False,undergraduate degree,-1.0,False,13.0,not specified,not yet
295247114,16-21,Employed,NaN,advanced (c1-c2),advanced (c1-c2),NaN,NaN,NaN,Not yet,not specified,NaT,NaN,Yes,NaN,eu citizen,NaN,Software Engineering,NaN,NaN,2024-03-18 17:06:57.105000+00:00,NaT,not requested,2024-03-18 17:08:03.937000+00:00,NaT,no sql,NaT,NaN,20785437789,690227426332,20785437789,paid,google,softwareentwickler,NaN,NaN,NaN,1,employed,not specified,False,False,0.000000,True,True,True,True,True,True,c1-c2,3,c1-c2,3,not specified,True,not specified,6.0,False,NaN,not specified,yes
295250351,56-64,Unemployed,NaN,mother tongue,advanced (c1-c2),NaN,NaN,NaN,NaN,not specified,NaT,NaN,Not yet,NaN,"permanent resident (""aufenthaltstitel"")",NaN,Orientation Course,NaN,NaN,2024-03-18 17:19:00.915000+00:00,NaT,not requested,2024-03-18 17:23:22.862000+00:00,NaT,no sql,NaT,NaN,1790797250849793,1793777382242322,NaN,paid,tiktok,1793512257382449,NaN,NaN,NaN,5,unemployed,not specified,False,False,0.000000,True,True,True,True,True,True,mother tongue,3,c1-c2,3,not specified,True,not specified,-1.0,False,NaN,not specified,not yet
295250966,25-40,Worker,NaN,beginner (a1-a2),intermediate (b1-b2),NaN,NaN,NaN,Not yet,not specified,NaT,NaN,Yes,NaN,eu citizen,NaN,Cybersecurity,NaN,NaN,2024-03-18 17:21:42.891000+00:00,NaT,not requested,2024-03-18 17:22:50.362000+00:00,NaT,no sql,NaT,NaN,20807334144,NaN,20807334144,paid,google,NaN,NaN,NaN,NaN,3,employed,not specified,False,False,0.000000,True,True,True,True,True,True,a1-a2,1,b1-b2,2,not specified,False,not specified,6.0,False,NaN,not specified,yes


In [22]:

states_data = {
    "Location": [
        "Baden-Württemberg", "Bayern (Bavaria)", "Berlin", "Brandenburg", "Bremen",
        "Hamburg", "Hessen (Hesse)", "Mecklenburg-Vorpommern", "Niedersachsen",
        "Nordrhein-Westfalen", "Rheinland-Pfalz", "Saarland", "Sachsen (Saxony)",
        "Sachsen-Anhalt", "Schleswig-Holstein", "Thüringen (Thuringia)",
        "Frankfurt (Hessen)", "Munich (Bayern)", "Aschaffenburg (Bayern)"
    ],
    "feat_unemployment_rate_pct": [
        3.7, 3.7, 9.7, 7.4, 11.1, 8.0, 5.3, 7.6, 5.5, 7.5, 5.6, 7.1, 6.0, 6.1, 5.8, 6.5, 5.3, 3.7, 3.7
    ],
    "feat_state_specific_funding_policies": [
        False, False, True, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False
    ],
    "feat_industry_demand_pct": [
        15, 18, 25, 10, 12, 20, 17, 8, 14, 19, 13, 9, 11, 9, 10, 8, 22, 23, 15
    ]
}

states_data = pd.DataFrame(states_data)


In [23]:
# Creating a dictionary to map the user-provided locations to the standardized locations used in the DataFrame
location_mapping = {
    'Niedersachsen (Lower Saxony)': 'Niedersachsen',
    'aschaffenburg': 'Aschaffenburg (Bayern)',
    'Saxony anhalt': 'Sachsen-Anhalt',
    'Saxony': 'Sachsen (Saxony)',
    'North Rhine-Westphalia': 'Nordrhein-Westfalen',
    'frankfurt': 'Frankfurt (Hessen)',
    'Munich': 'Munich (Bayern)',
    'Hessen (Hesse)': 'Hessen (Hesse)',
    'kampala': None,  # Not a German state
    'Ghana': None,  # Not a German state
    'Bavaria': 'Bayern (Bavaria)',
    'Saarland': 'Saarland',
    'Thüringen': 'Thüringen (Thuringia)',
    'Sachsen-Anhalt': 'Sachsen-Anhalt',
    'Bayern': 'Bayern (Bavaria)',
    'Baden-Württemberg': 'Baden-Württemberg',
    'Hessen': 'Hessen (Hesse)',
    'Schleswig-Holstein': 'Schleswig-Holstein',
    'Rheinland-Pfalz': 'Rheinland-Pfalz',
    'Bremen': 'Bremen',
    'Mecklenburg-Vorpommern': 'Mecklenburg-Vorpommern',
    'Hamburg': 'Hamburg',
    'Nordrhein-Westfalen': 'Nordrhein-Westfalen',
    'Niedersachsen': 'Niedersachsen',
    'Sachsen': 'Sachsen (Saxony)',
    'Brandenburg': 'Brandenburg',
    'Berlin': 'Berlin',
    'Nordrhein-Westfalen (North Rhine-Westphalia)': 'Nordrhein-Westfalen',
    'Bayern (Bavaria)': 'Bayern (Bavaria)',
    'Sachsen (Saxony)': 'Sachsen (Saxony)',
    "not specified":"not specified"
}


In [24]:
df['Location'] = df['state'].fillna("not specified").map(location_mapping)

In [25]:
merged_df = df.merge(states_data, on='Location', how="left")
merged_df.index = df.index

In [26]:
df = merged_df.copy()

In [27]:
df = preprocess_visa_status(df)



In [28]:
work_experience_mapping = {
    # Valid responses to "How long have you worked in Germany?"
    "More than 12 months": "More than 12 months",
    "Mehr als 12 Monate": "More than 12 months",  # German equivalent
    "I haven't worked in Germany": "I haven't worked in Germany",
    "Ich habe noch nie in Deutschland gearbeitet": "I haven't worked in Germany",  # German equivalent
    "Less than 6 months": "Less than 6 months",
    "Weniger als 6 Monate": "Less than 6 months",  # German equivalent
    "6-12 months": "6-12 months",
    "6 bis 12 Monate": "6-12 months",  # German equivalent

    # Responses that do not match the question → Categorized as "Other"
    "not specified": "not specified",
    "Abitur / Realschule": "Other",
    "Ausbildung": "Other",
    "Keinen": "Other",
    "B.A/B.Sc": "Other",
    "M.A/M.Sc": "Other",
    "Hauptschule": "Other",
    "Hauptschulabschluss": "Other",
    "PhD/MD": "Other",
    "Hauptschulabschluss + Musik Studium (Digitale-Musik Produktion, Schwehrpunkt Filmmusik) auf privater Ebene": "Other",
    "NO": "Other",
    "Yes": "Other",
    "Qualifizierte Hauptschulabschluss": "Other",
    "Hauptschulabschluss und Training als Pflegeassistenz": "Other",
    "Erweiterter Hauptschulabschluss": "Other",
    "leider nur hauptschul-abschluss": "Other",
    "QA": "Other",
    "Master im Betriebswirtschaftslehre": "Other",
    "Fachhochschulreife+Bankausbildung": "Other"
}


df["feat_work_experience_category"] = df["time_worked_in_de"].map(work_experience_mapping).fillna("not specified")


In [29]:
df['field_of_interest'].value_counts(dropna=False, normalize=True)

Data Analytics                           0.253233
Cybersecurity                            0.181253
AI Engineering                           0.164094
Software Engineering                     0.137249
Orientation Course                       0.080665
                                           ...   
DS                                       0.000010
Data+Analytics                           0.000010
Cybersecurity;Tech Orientation Course    0.000010
QA                                       0.000010
Web developer                            0.000010
Name: field_of_interest, Length: 167, dtype: float64

In [30]:
def field_of_intreset_preprocess(field):
    if pd.isna(field):
        return "not specified"
    field = field.strip().lower()
    if "oc" in field or "orientation" in field or "tech" in field:
        return "orientation course"
    if "cyber" in field:
        return "cybersecurity"     
    if "data" in field:
        return "data analytics"
    if "software" in field:
        return "software engineering"
    if "web" in field:
        return "web development"
    return "other"

df['feat_field_of_interest'] =df['field_of_interest'].apply(field_of_intreset_preprocess)
try:
    df = df.drop('field_of_interest_2', axis=1)
except KeyError:
    ...

In [31]:
df['feat_residents'] = df['residents'].fillna("not specified")

In [32]:
df["feat_days_to_mql"] = np.log1p((df['mql_date']-df['createdate']).dt.days)

In [33]:
df = preprocess_mql_date_features(df)
df = preprocess_mql_hour_minutes_features(df)

# here

In [34]:
# df['pre_feat_why_do_you_want_to_start_a_career_in_tech'] = df['why_do_you_want_to_start_a_career_in_tech'].fillna("_")

df[~df['why_do_you_want_to_start_a_career_in_tech'].isna()]['why_do_you_want_to_start_a_career_in_tech'].to_csv("table for chat.tsv")


age_range                                                                               NaN
employment_situation                                                             Unemployed
german_job_permit                                                                       NaN
english_level                                                                 C1 - Advanced
german_level                                                        B2 - Upper-intermediate
form_language                                                                           NaN
highest_level_of_education                                 Ausbildung / Vocational training
how_did_you_hear_about_us                                                     Search engine
in_contact_with_job_advisor                                                         Not yet
last_job                                                                            Factory
preferred_cohort                                                              No

In [35]:
def get_unique_words(text):
    if pd.isna(text):
        return 0
    else:
        return len(set(text.split(" ")))

def language_detection(text):
    if pd.isna(text ):
        return None
    else:
        try:
            lang = detect(text)
            
            if lang =="de":
                return lang
            else:
                return "en"
        except LangDetectException:
            return None

df['feat_length_of_answer'] =df['why_do_you_want_to_start_a_career_in_tech'].str.len()
df['feat_unique_words'] = df['why_do_you_want_to_start_a_career_in_tech'].apply(get_unique_words)

In [198]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

def build_tfidf_vectorizer(df, text_col='text'):
    """
    Takes a DataFrame with text and language columns,
    preprocesses the text by removing stopwords and lemmatizing
    for English and German texts, and then creates a TFIDF vectorizer.
    
    Parameters:
      df (pd.DataFrame): DataFrame containing the texts.
      text_col (str): Column name containing the text.
      lang_col (str): Column name containing the language info ('english', 'german', or None).
    
    Returns:
      vectorizer (TfidfVectorizer): Fitted TFIDF vectorizer.
      tfidf_matrix (sparse matrix): TFIDF-transformed matrix of the processed texts.
    """
    # Load spaCy models for English and German
    try:
        nlp_en = spacy.load("en_core_web_sm")
    except Exception as e:
        raise Exception("Please install the spaCy English model: en_core_web_sm") from e
        
    def process_text(row):
        text = row[text_col]
        
        # If language is not English or German, fallback to simple lowercasing
        
        if text and not pd.isna(text):
            return text.lower()
        else: 
            return ""
        
        # Choose the appropriate spaCy pipeline
        doc = nlp_de(text)
        
            
                   
        # Lemmatize, lower-case tokens, and remove stopwords & non-alphabetic tokens
        tokens = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
        return " ".join(tokens)
    
    # Create a new column with processed text
    df = df.copy()  # avoid modifying the original DataFrame
    df['processed_text'] = df.apply(process_text, axis=1)
    
    # Create and fit the TFIDF vectorizer on the processed text
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['processed_text'])
    
    return vectorizer, tfidf_matrix


    
vectorizer, tfidf_matrix = build_tfidf_vectorizer(final_answers, text_col='translation')
    
print("Feature names:", vectorizer.get_feature_names_out())
print("TFIDF matrix shape:", tfidf_matrix.shape)


Feature names: ['00' '000' '004917641825425' ... 'со' 'тоже' 'умею']
TFIDF matrix shape: (45103, 13361)


In [211]:
tfidf_dense = pd.DataFrame(tfidf_matrix.toarray(), index=final_answers.index)
tfidf_dense.columns = [f"tfidf_{i}" for i in range(tfidf_dense.shape[1])]

In [218]:
tfidf_dense.head()

,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,tfidf_10,tfidf_11,tfidf_12,tfidf_13,tfidf_14,tfidf_15,tfidf_16,tfidf_17,tfidf_18,tfidf_19,tfidf_20,tfidf_21,tfidf_22,tfidf_23,tfidf_24,tfidf_25,tfidf_26,tfidf_27,tfidf_28,tfidf_29,tfidf_30,tfidf_31,tfidf_32,tfidf_33,tfidf_34,tfidf_35,tfidf_36,tfidf_37,tfidf_38,tfidf_39,tfidf_40,tfidf_41,tfidf_42,tfidf_43,tfidf_44,tfidf_45,tfidf_46,tfidf_47,tfidf_48,tfidf_49,tfidf_50,tfidf_51,tfidf_52,tfidf_53,tfidf_54,tfidf_55,tfidf_56,tfidf_57,tfidf_58,tfidf_59,tfidf_60,tfidf_61,tfidf_62,tfidf_63,tfidf_64,tfidf_65,tfidf_66,tfidf_67,tfidf_68,tfidf_69,tfidf_70,tfidf_71,tfidf_72,tfidf_73,tfidf_74,tfidf_75,tfidf_76,tfidf_77,tfidf_78,tfidf_79,tfidf_80,tfidf_81,tfidf_82,tfidf_83,tfidf_84,tfidf_85,tfidf_86,tfidf_87,tfidf_88,tfidf_89,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99,tfidf_100,tfidf_101,tfidf_102,tfidf_103,tfidf_104,tfidf_105,tfidf_106,tfidf_107,tfidf_108,tfidf_109,tfidf_110,tfidf_111,tfidf_112,tfidf_113,tfidf_114,tfidf_115,tfidf_116,tfidf_117,tfidf_118,tfidf_119,tfidf_120,tfidf_121,tfidf_122,tfidf_123,tfidf_124,tfidf_125,tfidf_126,tfidf_127,tfidf_128,tfidf_129,tfidf_130,tfidf_131,tfidf_132,tfidf_133,tfidf_134,tfidf_135,tfidf_136,tfidf_137,tfidf_138,tfidf_139,tfidf_140,tfidf_141,tfidf_142,tfidf_143,tfidf_144,tfidf_145,tfidf_146,tfidf_147,tfidf_148,tfidf_149,tfidf_150,tfidf_151,tfidf_152,tfidf_153,tfidf_154,tfidf_155,tfidf_156,tfidf_157,tfidf_158,tfidf_159,tfidf_160,tfidf_161,tfidf_162,tfidf_163,tfidf_164,tfidf_165,tfidf_166,tfidf_167,tfidf_168,tfidf_169,tfidf_170,tfidf_171,tfidf_172,tfidf_173,tfidf_174,tfidf_175,tfidf_176,tfidf_177,tfidf_178,tfidf_179,tfidf_180,tfidf_181,tfidf_182,tfidf_183,tfidf_184,tfidf_185,tfidf_186,tfidf_187,tfidf_188,tfidf_189,tfidf_190,tfidf_191,tfidf_192,tfidf_193,tfidf_194,tfidf_195,tfidf_196,tfidf_197,tfidf_198,tfidf_199,tfidf_200,tfidf_201,tfidf_202,tfidf_203,tfidf_204,tfidf_205,tfidf_206,tfidf_207,tfidf_208,tfidf_209,tfidf_210,tfidf_211,tfidf_212,tfidf_213,tfidf_214,tfidf_215,tfidf_216,tfidf_217,tfidf_218,tfidf_219,tfidf_220,tfidf_221,tfidf_222,tfidf_223,tfidf_224,tfidf_225,tfidf_226,tfidf_227,tfidf_228,tfidf_229,tfidf_230,tfidf_231,tfidf_232,tfidf_233,tfidf_234,tfidf_235,tfidf_236,tfidf_237,tfidf_238,tfidf_239,tfidf_240,tfidf_241,tfidf_242,tfidf_243,tfidf_244,tfidf_245,tfidf_246,tfidf_247,tfidf_248,tfidf_249,tfidf_250,tfidf_251,tfidf_252,tfidf_253,tfidf_254,tfidf_255,tfidf_256,tfidf_257,tfidf_258,tfidf_259,tfidf_260,tfidf_261,tfidf_262,tfidf_263,tfidf_264,tfidf_265,tfidf_266,tfidf_267,tfidf_268,tfidf_269,tfidf_270,tfidf_271,tfidf_272,tfidf_273,tfidf_274,tfidf_275,tfidf_276,tfidf_277,tfidf_278,tfidf_279,tfidf_280,tfidf_281,tfidf_282,tfidf_283,tfidf_284,tfidf_285,tfidf_286,tfidf_287,tfidf_288,tfidf_289,tfidf_290,tfidf_291,tfidf_292,tfidf_293,tfidf_294,tfidf_295,tfidf_296,tfidf_297,tfidf_298,tfidf_299,tfidf_300,tfidf_301,tfidf_302,tfidf_303,tfidf_304,tfidf_305,tfidf_306,tfidf_307,tfidf_308,tfidf_309,tfidf_310,tfidf_311,tfidf_312,tfidf_313,tfidf_314,tfidf_315,tfidf_316,tfidf_317,tfidf_318,tfidf_319,tfidf_320,tfidf_321,tfidf_322,tfidf_323,tfidf_324,tfidf_325,tfidf_326,tfidf_327,tfidf_328,tfidf_329,tfidf_330,tfidf_331,tfidf_332,tfidf_333,tfidf_334,tfidf_335,tfidf_336,tfidf_337,tfidf_338,tfidf_339,tfidf_340,tfidf_341,tfidf_342,tfidf_343,tfidf_344,tfidf_345,tfidf_346,tfidf_347,tfidf_348,tfidf_349,tfidf_350,tfidf_351,tfidf_352,tfidf_353,tfidf_354,tfidf_355,tfidf_356,tfidf_357,tfidf_358,tfidf_359,tfidf_360,tfidf_361,tfidf_362,tfidf_363,tfidf_364,tfidf_365,tfidf_366,tfidf_367,tfidf_368,tfidf_369,tfidf_370,tfidf_371,tfidf_372,tfidf_373,tfidf_374,tfidf_375,tfidf_376,tfidf_377,tfidf_378,tfidf_379,tfidf_380,tfidf_381,tfidf_382,tfidf_383,tfidf_384,tfidf_385,tfidf_386,tfidf_387,tfidf_388,tfidf_389,tfidf_390,tfidf_391,tfidf_392,tfidf_393,tfidf_394,tfidf_395,tfidf_396,tfidf_397,tfidf_398,tfidf_399,tfidf_400,tfidf_401,tfidf_402,tfidf_403,tfidf_404,tfidf_405,tfidf_406,tfidf_407,tfidf_408,tfidf_409,tfidf_410

In [36]:
def multiply_plans_by_value(plan):
    if pd.isna(plan):
        return 0
    if plan in ("7 Months", "8 Months"):
        return 29600
    if plan == "1 Month":
        return 3900
    else:
        return 46540
    

df['category_label_bg_enrolled'] = df['closed_won_deal__program_duration'].replace("7 Months", "8 Months").fillna("not enrolled")
df['regression_label'] = df['closed_won_deal__program_duration'].apply(multiply_plans_by_value)

In [37]:
df['category_label_bg_enrolled'].value_counts()

not enrolled    98661
14 Months         550
8 Months          527
1 Month            95
Name: category_label_bg_enrolled, dtype: int64

In [39]:
import random

In [121]:
trans_data = pd.read_csv('output_checkpoint.csv')
trans_data_1 = pd.read_csv('output_checkpoint_1.csv')
trans_data_2 = pd.read_csv('output_checkpoint_2.csv')
trans_data_3 = pd.read_csv('output_checkpoint_3.csv')

# 



trans_data = trans_data.set_index("index")
trans_data_1 = trans_data_1.set_index("index")
trans_data_2 = trans_data_2.rename(columns={"Unnamed: 0":"index"}).set_index("index")
trans_data_3 = trans_data_3[~trans_data_3['language'].isna()].set_index("index")

C:\Users\Tal\AppData\Local\Temp\ipykernel_23620\3979741556.py:3: DtypeWarning: Columns (3,12,18,33) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_data_2 = pd.read_csv('output_checkpoint_2.csv')


In [122]:
trans_data_2['why_do_you_want_to_start_a_career_in_tech'].loc[102098279515]

'Ich sehe meine potenziell dazu '

In [123]:
idiot = pd.concat([trans_data, trans_data_1, trans_data_2,trans_data_3])

In [124]:
idiot['language']= idiot['language'].fillna(idiot['language_detected'])
idiot['translation'] = idiot['translation'].fillna(idiot['translated_to_english'])
idiot = idiot[['language','translation']].copy()

In [125]:
with_translations = idiot.join(new_df[['why_do_you_want_to_start_a_career_in_tech']], how="left")

In [126]:
with_translations.loc[7420801]

language                                                                                 en
translation                                                                             NaN
why_do_you_want_to_start_a_career_in_tech    Because its the best selling market place now.
Name: 7420801, dtype: object

In [134]:
with_translations['language'] = with_translations['language'].str.replace("German", "de").str.replace("und", "unknown")

In [135]:
with_translations= with_translations.reset_index().drop_duplicates().set_index('index')

In [136]:
with_translations['translation'] = np.where(
    with_translations['language']=='en',
    with_translations['translation'].fillna(with_translations['why_do_you_want_to_start_a_career_in_tech']), 
                                            with_translations['translation'])

In [152]:
with_translations.reset_index()['index'].value_counts().head(15)

65786874386    3
52635775568    2
51684749589    2
51687718502    2
51691206684    2
51696156802    2
51697213724    2
51697263914    2
51698870143    2
51709683793    2
51713143120    2
54100666966    2
51714980353    2
51715526449    2
51716731549    2
Name: index, dtype: int64

In [156]:
with_translations = with_translations.dropna(thresh=2)

In [157]:
with_translations.reset_index()['index'].value_counts().head(15)

68337190760    2
258903450      2
62231850870    2
266046548      2
288323751      2
280237854      2
62244400445    2
266017130      2
62247019612    2
66495477363    2
58050784790    2
288269058      2
62278318512    2
70284423856    2
70285409126    2
Name: index, dtype: int64

In [166]:
with_translations = with_translations.reset_index().drop_duplicates(['index', 'why_do_you_want_to_start_a_career_in_tech','language']).set_index("index")

In [195]:
# with_translations.loc[]

final_answers = pd.concat([with_translations.loc[doubles[doubles['index']==1].index], double_to_single])



In [225]:
final_answers[final_answers['translation'].isna()].head(60)

final_answers.loc[292607361]

language                                                                  en
translation                                  Good opportunity for a good job
why_do_you_want_to_start_a_career_in_tech    Good opportunity for a good job
Name: 292607361, dtype: object

In [191]:
doubles = with_translations.reset_index()['index'].value_counts().to_frame()

double_to_single = with_translations.loc[doubles[doubles['index']>1].index].sort_values('language').reset_index().drop_duplicates(subset=["index", 'why_do_you_want_to_start_a_career_in_tech'], keep="last").set_index("index")

In [226]:
final_answers.to_csv("all the answers translated.tsv",sep="\t")

In [ ]:
idiot.to_csv("All translated answers.tsv", sep='\t', index=True)

In [ ]:
idiot

In [ ]:
idiot['translation'] = idiot['translation'].str.replace("'text': '","")

In [ ]:
idiot['translation'].loc[254792952]

In [ ]:
idiot.loc[102098279515]

In [ ]:
idiot.join(df, how="left")[['language', 'translation','why_do_you_want_to_start_a_career_in_tech']].loc[102098279515]

In [ ]:
df['why_do_you_want_to_start_a_career_in_tech'].loc[102098279515]

In [ ]:
translation_comparison = some_translations.merge(df[['why_do_you_want_to_start_a_career_in_tech']].reset_index(), left_on='index', right_on='id', how="left").drop(columns=['id'])


In [ ]:
equals = translation_comparison[translation_comparison['translation']==translation_comparison['why_do_you_want_to_start_a_career_in_tech']]
not_equals = translation_comparison[translation_comparison['translation']!=translation_comparison['why_do_you_want_to_start_a_career_in_tech']].copy()

In [ ]:
translation_comparison['translation'].fillna(translation_comparison)

In [ ]:
trans_df.loc[102098279515]#['why_do_you_want_to_start_a_career_in_tech']

In [ ]:
trans_df = df[~df['why_do_you_want_to_start_a_career_in_tech'].isna()].copy()


In [ ]:
trans_df_resetted = trans_df.reset_index()

In [ ]:
trans_df_resetted[~trans_df_resetted['id'].isin(idiot['index'].values)]

In [ ]:
trans_df = df[~df['why_do_you_want_to_start_a_career_in_tech'].isna()].copy()



In [ ]:
idiot

In [ ]:
df

In [ ]:
len(trans_df)

In [ ]:
import json
import pandas as pd
import time
from openai import OpenAI

def detect_and_translate(text: str, api_key: str):
    client = OpenAI(api_key=api_key)
    
    for _ in range(3):  # Retry mechanism
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                response_format={
                    "type": "json_object"
                  },
                messages=[
                    {"role": "system", "content": "You are a language detection and translation assistant."},
                    {"role": "user", "content": f"""
                        Detect the language of the following text and translate it to English if necessary.

                        Text: "{text}"

                        Respond strictly in the following JSON format:
                        {{
                            "language": "detected_language",
                            "translation": "translated_text"
                        }}

                        If the language is English, return:
                        {{
                            "language": "en",
                            "translation": null
                        }}
                    """}
                ],
                temperature=0,
                max_tokens=1500
            )
            time.sleep(0.1)
            response_text = response.choices[0].message.content.strip()
           
            return json.loads(response_text)
        except Exception as e:
            print(f"API call failed: {e}, retrying...")
            time.sleep(2)
    
    return {"text":text, "error": "API request failed after retries"}

def process_dataframe(test, key, output_file="output_checkpoint_3.csv", checkpoint=1400):
    results = []
    
    for i, (idx, text) in enumerate(test['why_do_you_want_to_start_a_career_in_tech'].items()):
        result = detect_and_translate(text, key)
        result["index"] = idx  # Keep original index
        results.append(result)
        
        if (i + 1) % checkpoint == 0:
            print(f"Checkpoint reached: {i + 1} rows processed, saving intermediate results...")
            df_checkpoint = pd.DataFrame(results).set_index("index")
            df_checkpoint.to_csv(output_file)
    
    df_final = pd.DataFrame(results).set_index("index")
    df_final.to_csv(f"pre_{output_file}")
    test = test.join(df_final[["language", "translation"]])
    test.rename(columns={"language": "language_detected", "translation": "translated_to_english"}, inplace=True)
    test.to_csv(output_file)  # Save final results with index
    return test


In [ ]:
translated_texts = process_dataframe(trans_df, key)


In [ ]:
translated_texts

In [ ]:
def process_dataframe(test, key):
    test[['language_detected', 'translated_to_english']] = test['why_do_you_want_to_start_a_career_in_tech'].apply(
        lambda x: pd.Series(detect_and_translate(x, key))
    )
    return test

In [ ]:
pwd

In [ ]:
trans_df.loc[292857672]['why_do_you_want_to_start_a_career_in_tech']